In [ ]:
#default_exp poker

# Lesson 1 + Homework from the Udacity CS212 Course - Design of Computer Programs by Peter Norvig

This was one of the first courses released by Udacity back in 2012. I took the course at the time and struggled *a lot* to be able to complete the programming assignments.
However, over the years my familiarity with Python and my general programming chops have improved and I figured it was time to give it another try. My motivation to do this again is also motivated by the Advent Of Code 2020 challenge, and I know the skills taught in this course will greatly improve my ability to complete similar coding challenges.

Below is the code for the Poker program taught in Lesson 1. This code is a merger of what I wrote in the quizzes and I merged in some ideas from Peter's solution. I am not going to explain all of this code below, and I encourage you to do the free course if you are interested in this code.

### Poker

In [2]:
#export

from random import shuffle

mydeck = [r+s for r in '23456789TJQKA' for s in 'SHDC']

def deal(numhands, n=5, deck=mydeck):
    "Deal numhands poker hands"
    shuffle(deck)
    return [deck[n*i:n*(i+1)] for i in range(numhands)]

def poker(hands):
    "Return the best hand: poker([hand,...]) => [hand, ...]"
    return allmax(hands, key=hand_rank)

def allmax(iterable, key=None):
    "return a list of all items equal to the max of the iterable"
    key = key or (lambda x: x)
    best = max(iterable, key=key)
    return [item for item in iterable if key(item)==key(best)]

def hand_rank(hand):
    "Returns the rank of a hand"
    ranks = card_ranks(hand)
    if straight(ranks) and flush(hand):            # straight flush
        return (8, max(ranks))
    elif kind(4, ranks):                           # 4 of a kind
        return (7, kind(4, ranks), kind(1, ranks))
    elif kind(3, ranks) and kind(2, ranks):        # full house
        return (6, kind(3, ranks), kind(2, ranks))
    elif flush(hand):                              # flush
        return (5, ranks)
    elif straight(ranks):                          # straight
        return (4, max(ranks))
    elif kind(3, ranks):                           # 3 of a kind
        return (3, kind(3, ranks), ranks)
    elif two_pair(ranks):                          # 2 pair
        return (2, two_pair(ranks), ranks)
    elif kind(2, ranks):                           # kind
        return (1, kind(2, ranks), ranks)
    else:                                          # high card
        return (0, ranks)

def kind(n, ranks):
    "For a given int n, return the rank of card that has that kind or False"
    for r in ranks:
        if ranks.count(r)==n: return r
    return None

def two_pair(ranks):
    """
    (ranks,) => (int, int)
    For a given set of card ranks, return rank of pairs if two pairs, else None
    """
    p1 = kind(2, ranks)
    if p1:
        p2 = kind(2, [r for r in ranks if r!=p1])
    if p1 and p2:
        return (p1, p2)
    return None

def card_ranks(hand):
    """
    (str,) => (int,)
    Return rank of cards for a hand
    """
    ranks = ['--23456789TJQKA'.index(r) for r,s in hand]
    ranks.sort(reverse=True)
    return [5,4,3,2,1] if (ranks == [14, 5, 4, 3, 2]) else ranks

def straight(ranks):
    "Returns True if hand is a straight, otherwise False"
    return len(set(ranks))==5 and (max(ranks)-min(ranks) == 4)

def flush(hand):
    "Return True if all cards have the same suit, otherwise False"
    suit = hand[0][1]
    return all(s==suit for r,s in hand)

In [5]:
def test():
    "Test cases for the functions in the poker program"
    sf = "6C 7C 8C 9C TC".split() # straight flush
    fk = "9D 9H 9S 9C 7D".split() # 4 of a kind
    fh = "TD TC TH 7C 7D".split() # full house
    tp = "5S 5D 9H 9C 6S".split() # 2 pair
    al = "AC 5D 4D 3H 2S".split() # straight aces low (special case)
    na = "2C 4C 5H 7D TS".split() # nothing, 10 high
    
    fkranks = card_ranks(fk)
    tpranks = card_ranks(tp)
    alranks = card_ranks(al)
    naranks = card_ranks(na)
    
    assert kind(4, fkranks) == 9
    assert kind(4, tpranks) == None
    assert kind(2, fkranks) == None
    assert kind(1, fkranks) == 7
    assert two_pair(tpranks) == (9,5)
    
    assert straight([9,8,7,6,5]) == True
    assert straight([9,8,8,6,5]) == False
    assert straight(alranks) == True
    assert flush(sf) == True
    assert flush(fk) == False
    
    assert card_ranks(sf) == [10, 9, 8, 7, 6]
    assert card_ranks(fk) == [9, 9, 9, 9, 7]
    assert card_ranks(fh) == [10, 10, 10, 7, 7]
    assert card_ranks(na) == [10, 7, 5, 4, 2]
    
    assert hand_rank(sf) == (8, 10)
    assert hand_rank(fk) == (7, 9, 7)
    assert hand_rank(fh) == (6, 10, 7)
    assert hand_rank(na) == (0, [10, 7, 5, 4, 2])
    
    assert poker([sf, fk, fh]) == [sf]
    assert poker([fk, fh]) == [fk]
    assert poker([fh, fh]) == [fh, fh]
    assert poker([fh]) == [fh]
    assert poker([sf] + [fh]*99)  == [sf]
    assert poker([na, al, tp]) == [al]
        
    return "test pass"
                 
print(test())

test pass


# Homework Assignment

The homework assignment has two parts:
1) Given a 7 card hand, determine the best 5 card hand

2) Given a 7 card hand *with red or black wild cards*, deterime the best 5 card hand.

### Part 1

This is a relatively simple solution since we can use the itertools.combinations function
to generate every possible 5-card combination from our 7 cards, then use the existing
hand_rank function to pick the best one. My solution was nearly identical to Peter's

In [166]:
#export
import itertools

def best_hand(hand):
    hands = itertools.combinations(hand, 5)
    return max(hands, key=hand_rank)

## Part 2

This part is significantly more complex, and the case where a hand may have a red *and* and black wild card was the most challenging for me.
Although my solution passes all of the test cases, it is not the most elegant solution (especially compared to Peter's, below)

Another itertools function that greatly helped with my solution was itertools.product which returns every combination of elements selected from a list of lists.
I struggled with the idea of how to ensure we are selecting only 1 card from each wild card list and the remaining cards from the hand we were dealt, but I was create a variable `num_wild` that I used to determine how many wild cards were found.

In [258]:
#export
all_ranks = '23456789TJQKA'
black_cards = [r+s for r in all_ranks for s in 'SC']
red_cards = [r+s for r in all_ranks for s in 'HD']

def best_wild_hand(hand):
    "Try all values for jokers in all 5-card selections."
    hands = []
    wild_cards = []
    num_wild=0
    if '?B' and '?R' in hand:
        hand.remove('?B')
        hand.remove('?R')
        num_wild=2
        red_cards2 = [c for c in red_cards if c not in hand]
        black_cards2 = [c for c in black_cards if c not in hand]
        wild_cards = [c for c in itertools.product(red_cards2, black_cards2)]
    elif '?B' in hand:
        hand.remove('?B')
        num_wild=1
        wild_cards = [(c,) for c in black_cards if c not in hand]
    elif '?R' in hand:
        hand.remove('?R')
        num_wild=1
        wild_cards = [(c,) for c in red_cards if c not in hand]
        
    hands = [c for c in itertools.combinations(hand, 5)]
    hands.extend([comb+wild for comb in itertools.combinations(hand, 5-num_wild) for wild in wild_cards])
    
    return max(hands, key=hand_rank)

In [ ]:
def test_best_wild_hand():
    assert (sorted(best_wild_hand("6C 7C 8C 9C TC 5C ?B".split()))
            == ['7C', '8C', '9C', 'JC', 'TC'])
    assert (sorted(best_wild_hand("TD TC 5H 5C 7C ?R ?B".split()))
            == ['7C', 'TC', 'TD', 'TH', 'TS'])
    assert (sorted(best_wild_hand("JD TC TH 7C 7D 7S 7H".split()))
            == ['7C', '7D', '7H', '7S', 'JD'])
    return 'test_best_wild_hand passes'

print(test_best_wild_hand())

In [ ]:
#export
def best_wild_hand_peter(hand):
    hands = [best_hand(h) for h in itertools.product(*map(replacement, hand))]
    best = max(hands, key=hand_rank)
    print(best)
    return best
    
def replacement(card):
    """str => [str,..]
    If the card is a wild card, replaces it with the list of cards it can replace,
    otherwise return the card in a list
    """
    if card == '?R': return red_cards
    elif card == '?B': return black_cards
    else: return [card]

In [260]:
def test_best_wild_hand_peter():
    assert (sorted(best_wild_hand_peter("6C 7C 8C 9C TC 5C ?B".split()))
            == ['7C', '8C', '9C', 'JC', 'TC'])
    assert (sorted(best_wild_hand_peter("JD TC TH 7C 7D 7S 7H".split()))
            == ['7C', '7D', '7H', '7S', 'JD'])
    assert (sorted(best_wild_hand_peter("TD TC 5H 5C 7C ?R ?B".split()))
            == ['7C', 'TC', 'TD', 'TH', 'TS'])
    return 'test_best_wild_hand_peter passes'

print(test_best_wild_hand_peter())

('7C', '8C', '9C', 'TC', 'JC')
('JD', '7C', '7D', '7S', '7H')
('TD', 'TC', '7C', 'TH', 'TS')
test_best_wild_hand_peter passes
